# Сессия 4

---

In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns=50

### Импорт моделей

Для импорта моделей все так же импользую pickle

In [2]:
import pickle

Импортирую MinMaxScaler

In [3]:
#Импорт модели из файла
scaler = pickle.load(open('models/scaler.sav', 'rb'))

Теперь имортирую модель алгоритма KNN

In [4]:
#Импорт модели из файла
model = pickle.load(open('models/neigh.sav', 'rb'))

### 4.1 Разработка API

импортирую дата сет помошник, в котором содержится вся информация по кодам категорям и описанием товаров категорий, и на его основе буду строить рекомендательную систему

In [5]:
#Импорт вспомогательного набора данных
healper = pd.read_csv("healper.csv", encoding="utf-8-sig")

Список для получения релевантных предложений

In [6]:
#В этом списке будут находится значения 0 или 1, означающие нравится ли это пользователю или нет
relevant = []

API разработаю в виде класса

In [7]:
class API:
    #Метод инициализации буждет пустой
    def __init__(self):
        #pass нужен для того, чтобы я мог оставить этот блок пустым и не выдавало ошибок
        pass
    
    #Метод, который будет выдавать предсказание с ручного ввода
    def predict(X):
        try:
            #Получаю кластер
            cluster = model.predict(scaler.transform(X))[0]
            #На его основе буре топ популярных категорий продавца
            code = healper[healper["CLUSTERNAME"] == cluster]["MCC_CODE"].value_counts().head(3).index[0]
            #Возвращаю топ самых популярных магазинов и оставляю 3
            return healper[healper["MCC_CODE"] == code][["NAME", "MCC_NAME","MCC_DESCRIPTION"]].value_counts().head(3).index
        except:
            #На его основе буре топ популярных категорий продавца
            code = healper[healper["CLUSTERNAME"] == X]["MCC_CODE"].value_counts().head(3).index[0]
            #Возвращаю топ самых популярных магазинов и оставляю 3
            return healper[healper["MCC_CODE"] == code][["MCC_NAME","MCC_DESCRIPTION"]].value_counts().head(3).index
            
        
       
    #Метод для чтения файла из csv файла
    def csv(url):
        #Читаю данные из url
        df = pd.read_csv(url, encoding="utf-8-sig")
        #Оставляю только нужные и помещаю их в переменную X
        X = df[["MCC_CODE", "SUM_TRANS", "REG_CODE"]]
        #Использую метод предсказания для каждого атрибута
        df["PREDICTED"] = model.predict(scaler.transform(X))
        #И на основе предсказания делаю рекомендацию
        df["RECOMENDATION"] = df["PREDICTED"].apply(lambda x: list(API.predict(x)))
        #В конце сохраняю файл в другой csv
        df.to_csv("predicted.csv", encoding="utf-8-sig", index=False)
        #Вывожу уведомление для пользователя, о том, что все прошло успешно
        return "Вашь файл с метками называется - predicted.csv"
    
    #Метода для отметки релевантных предложений
    def reval(point):
        relevant.append(point)

#### Тестировочные данные

Данные для ввода csv:<br>C:\Users\student\Desktop\Fedulov\test\dt.csv

Данные для ввода с руки:<br>541 520 32

### 4.2 RecomendationSystemV1

Теперь займусь написанием программного интерфейса

Он будет представлять из себя консольное приложение для взаимодействия пользователя с API

In [8]:
q = ""
#Список команд
print("\nКомманды:\n 1)Ручной - при вводе этой комманды необходимо ввести данные для предсказания в поле ввода,\nна выходе будет определена категория покупки\n\n 2)Csv - при вводе этой комманды, необходимо  указать путь к csv файлу, на выходе вы получите уведомление о том,\nчто был создан новый файл (файл csv с новым атрибутом, который означет категорию покупки)\n\n 3)Команды - выведет список команд\n\n 4)Выход - завершает работу программы")
#Цикл в котором будет реализовано взаимодействие с пользователем
while q != "выход":
    #Обработка ошибок (если все хорошо)
    try:
        #Строка для ввода комманд
        q = input("Введите команду - ")
        #привожу к нижнему регистру
        q = q.lower().strip()
        #Команда ручного ввода
        if q == "ручной":
            #Ввод значенй
            z = input("введите код категории продовца, сумму транзакции и регион - ")
            #обработка значений
            z = z.split(" ")
            #Заполняю список обработанными числовыми значениями пользоваетлей
            X = [[int(i) for i in z]]
            #Вывод рекомендаций
            print("\nВам рекомендуются следующие магазины:")
            #Использование функции API
            for i in  API.predict(X):
                #Этот метод возвращает кортеж и поэтому, могу обратится к нему по индексу
                print(f"{i[0]}\n{i[1]}\n{i[2]}\n")
            #Теперь запрошу у пользователя оценку рекомендации
            point = int(input("Оцениете рекомендацию от 0 или 1 - "))
            #Добавленеи оценки
            if point == 1 or point == 0:
                API.reval(point)
            else:
                continue
        #Функция для обработки csv файлов
        elif q == "csv":
            #получение пути файла
            url = input("Укажите путь к файлу - ")
            #Использование функции API для csv файлов
            print(API.csv(url))
        #Вывод отчета о поставленых оценках
        elif q == "отчет":
            print("Вы ставили следующие оценки рекомендациям:")
            #Переформатировал вывод вместо 1 или 0 на хорошо или плохо
            for i in relevant:
                #Простое условие
                if i == 1:
                    print("Хорошо")
                else:
                    print("Плохо")
        #Функция для вывода комманд
        elif q == "команды":
            print("\nКомманды:\n 1)Ручной - при вводе этой комманды необходимо ввести данные для предсказания в поле ввода,\nна выходе будет определена категория покупки\n\n 2)Csv - при вводе этой комманды, необходимо  указать путь к csv файлу, на выходе вы получите уведомление о том,\nчто был создан новый файл (файл csv с новым атрибутом, который означет категорию покупки)\n\n 3)Команды - выведет список команд\n\n 4)Выход - завершает работу программы")      
    #Обработка ошибок (если что то пошло не так)
    except:
        #вывод об уведомлении некорректного ввода
        print("Ошибка ввода, повторите попытку, следуя инструкции")


Комманды:
 1)Ручной - при вводе этой комманды необходимо ввести данные для предсказания в поле ввода,
на выходе будет определена категорию покупки

 2)Csv - при вводе этой комманды, необходимов  указать путь к csv файлу, на выходе вы получите уведомление о том,
что был создан новый файл (файл csv с новым атрибутом, который означет кактегорию покупки)

 3)Команды - выведет список команд

 4)Выход - завершает работу программы
Введите команду - Ручной
введите код категории продовца, сумму транзакции и регион - 541 520 32

Вам рекомендуются следующие магазины:
pyaterochka
Бакалейные магазины, супермаркеты
Торговые точки, которые продают полную линейку продуктов питания для домашнего потребления. Пищевые продукты для продажи включают бакалейные товары, мясо, продукты, молочные продукты и консервированные, замороженные, предварительно упакованные и сухие продукты. Также продукты для продажи могут включать ограниченный выбор посуды, чистящих и полирующих изделий, средств личной гигиены, косм

После того как csv файл быд обработан, выведу содержимое для одного значения

In [9]:
#Чтение данных
prd = pd.read_csv("predicted.csv", encoding="utf-8-sig")

In [10]:
#Вывел присвоенный класс
prd["PREDICTED"][0]

'Обычная покупка'

In [11]:
#Выведу названия предприятия
prd["RECOMENDATION"][0].split("'")[1]

'Бакалейные магазины, супермаркеты'

In [12]:
#Выведу товары и услуги предоставляемые даннм торговым предприятием
prd["RECOMENDATION"][0].split("'")[3]

'Торговые точки, которые продают полную линейку продуктов питания для домашнего потребления. Пищевые продукты для продажи включают бакалейные товары, мясо, продукты, молочные продукты и консервированные, замороженные, предварительно упакованные и сухие продукты. Также продукты для продажи могут включать ограниченный выбор посуды, чистящих и полирующих изделий, средств личной гигиены, косметики, поздравительных открыток, книг, журналов, предметов домашнего обихода и сухих товаров. Эти точки также могут управлять специализированными отделами, такими как лавка деликатесов, мясная лавка, аптека или цветочный отдел.\\nДля магазинов, которые продают ограниченный выбор продуктов или предметов специальности, используется MCC 5499.'

### 4.3 Программная документация

Для разработанного приложения и API, программная документация и руководство пользователя находится в архиве с проектом

### 4.4 Отчет

В ходе рабоыт было выполнено:
* 4.1 Разработка API - Был разработан интерфейс, позволяющий подключить разработанный программный продукт в качестве модуля к системе; Был обеспечен сбор и хранение информации о товарах с открытых источников, которые могут быть рекомендованы к покупке пользователям; Интерфейс содержит документацию по использованию;
* 4.2 Разработка приложения - Было разработано приложение рекомендательной системы, которое выполняет действия на основе API; Приложение выполняет следующие действия: ввод параметров для поиска рекомендаций, загрузка параметров для поиска из файла .csv, содержащего информацию о пользователе, по введенным параметрам выводить рекомендованные товары и услуги и возможность отмечать релевантные предложения («лайки»);
* 4.3 Программная документация - Была разработанна программная документация и руководство пользователя для API и разработанного приложения;